In [1]:
import sqlite3
import numpy as np
import pandas as pd
import ctypes
import shutil
import os
MessageBox = ctypes.windll.user32.MessageBoxW
from System_Assessment_Generate_Models_Variables import *


In [2]:
def sql_to_df(sql,model):
    con = sqlite3.connect(model)
#     print(sql)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
#             print(sql)
            cur.execute(sql)
    else:
        sql = sqls
#         print(sql)
        cur.execute(sql)
    cur.close()
    con.commit()
    con.close()
  
def df_to_sql(df,table_name,model):
    conn = sqlite3.connect(model)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    
def generate_script(script_path,mu_path):  
    script_path_new = os.path.splitext(mu_path)[0] + '.cs'
    shutil.copy(script_path, script_path_new)
    
def generate_mupp(model_original,mu_path):
    mupp_path_original = os.path.splitext(model_original)[0] + '.mupp'
    mupp_path_new = os.path.splitext(mu_path)[0] + '.mupp'   
    with open(mupp_path_original, 'r') as source_file, open(mupp_path_new, 'w') as output_file:
        for line in source_file:
            if line.startswith(r'   DBFilePath = |.'):
                output_file.write('   DBFilePath = |.\\' + os.path.splitext(os.path.basename(mu_path))[0] + '.sqlite|\n')
            else:
                output_file.write(line)
    

In [3]:
#Early error catching
if generate_bsf:

    for gwi in gwis:
        if np.mod(gwi,11.2) > 0:

            message = 'GWI ' + str(gwi) + ' is not a multiple of 11.2\n\nContinue?'

            if MessageBox(None, message, 'Warning', 4) == 7:
                MessageBox(None, "Please correct gwis", 'Info', 0)
                raise ValueError(message)
            else:
                pass

    

In [4]:
if generate_future:
    
    mu_paths = []

    keep_cols = ['Catchment','Zone','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Area_Com','Area_Ind','Area_Inst']
    pop_df = pd.read_excel(population_sheet,sheet_name=population_tab,usecols=keep_cols,dtype={'Catchment': str})

    # pop_df = pop_df[pop_df.Catchment!='84394S']
    # pop_df = pop_df[pop_df.Catchment!='84279']
    # pop_df = pop_df.drop(pop_df.index[-1])

    # Melt the DataFrame to combine the columns
    columns_to_combine = ['Pop_ResLD', 'Pop_ResHD', 'Pop_Mixed', 'Area_Com', 'Area_Ind', 'Area_Inst']
    pop_df_melt = pd.melt(pop_df, id_vars=['Catchment', 'Zone', 'Year'], value_vars=columns_to_combine, var_name='Type', value_name='Value')
    pop_df_melt[['Major_type', 'Minor_type']] = pop_df_melt['Type'].str.split('_', n=1, expand=True)
    pop_df_melt = pop_df_melt.drop(columns=['Type'])
    pop_df_melt['MUID'] = pop_df_melt.Catchment + '_' + pop_df_melt.Minor_type
    pop_df_melt

    #Check for missing catchments
    catchment_years = []
    sql = "SELECT catchmentid FROM msm_Loadpoint GROUP BY catchmentid"
    muids = list(sql_to_df(sql,model_original).catchmentid)

    for muid in muids:
        for year_scenario in year_scenario_list:
            year = year_scenario[0]
            catchment_years.append([muid,year])
    catchment_year_df = pd.DataFrame(catchment_years,columns=(['Catchment','Year']))
    catchment_year_df 

    merged = catchment_year_df.merge(pop_df[['Catchment', 'Year']], on=['Catchment', 'Year'], how='left', indicator=True)

    not_founds = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

    if len(not_founds) > 0:
        message = "WARNING.The following catchment/year combinations are not found\n\n"
        for index, row in not_founds.iterrows():
            message += row[0] + ', ' + str(row[1]) + '.\n'
        message += '\nContinue?'

        if MessageBox(None, message, 'Warning', 4) == 7:
            MessageBox(None, "Please report the missing catchment(s)", 'Info', 0)
            raise ValueError(message)
        else:
            pass
        
    #Generate runoff hotstart model without C#
    model_name = model_area + '_Runoff_Hotstart.sqlite'
    mu_path = output_folder + "\\" + model_name    
    os.remove(mu_path) if os.path.exists(mu_path) else None
    shutil.copyfile(model_original, mu_path)
    generate_mupp(model_original,mu_path)

    for year_scenario in year_scenario_list:
        year = year_scenario[0]
        scenario = year_scenario[1]
        turnons = year_scenario[2]

        model_name = model_area + '_' + str(year) + 'pop_V' + str(version) + '.sqlite'

        if os.path.basename(model_original) == model_name:

            message = "Tool ends. For year " + str(year) + ", the new model name '" + model_name + "' is the same as the original."
            MessageBox(None, message, 'Info', 0)
            raise ValueError("message")

        #Delete sqlite, mupp and cs if they exist and create new
        mu_path = output_folder + "\\" + model_name
        
        mu_paths.append(mu_path)

        generate_script(script_path,mu_path)
        generate_mupp(model_original,mu_path)

        os.remove(mu_path) if os.path.exists(mu_path) else None
        shutil.copyfile(model_original, mu_path)

        sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
        muids = list(sql_to_df(sql,mu_path).muid)
        for muid in muids:
            muid_new = muid.replace(str(year_original) + 'p',str(year) + 'p')
            sql = "UPDATE msm_Project SET muid = '" + muid_new + "' WHERE muid = '" + muid + "'"
            execute_sql(sql, mu_path)

            sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"
            execute_sql(sql, mu_path)

        sql = "UPDATE msm_Project SET IncludeToBatchNo = 0 WHERE MUID LIKE '%h-AES_%'"
        execute_sql(sql, mu_path)
        
        sql = "UPDATE msm_Project SET scenarioname = '" + scenario + "' WHERE MUID NOT LIKE '%h-AES_%' AND enable_hd = 1"
        execute_sql(sql, mu_path)

        for turnon in turnons:
            sql = "UPDATE msm_Project SET IncludeToBatchNo = 1 WHERE MUID LIKE '%h-AES_%' AND MUID LIKE '%" + turnon + "%' "
            sql += "AND SUBSTR(scenarioname,1," + str(len(scenario)) + ") = '" + scenario  + "'"
            execute_sql(sql, mu_path)

        pop_df_melt_year = pop_df_melt[pop_df_melt.Year==year]

        df_to_sql(pop_df_melt_year,'New_Population',mu_path)

        sql = "UPDATE msm_Loadpoint SET Population = "
        sql += "(SELECT Value FROM New_Population WHERE MUID = msm_Loadpoint.muid AND Major_Type = 'Pop')"
        execute_sql(sql, mu_path)

        sql = "UPDATE msm_Loadpoint SET ICIArea = "
        sql += "(SELECT Value FROM New_Population WHERE MUID = msm_Loadpoint.muid AND Major_Type = 'Area')"
        execute_sql(sql, mu_path)

        sql = "UPDATE msm_Loadpoint SET loadflow = PerCapitaLoad * Population / 86400 WHERE LoadCategory = 'Mixed' OR LoadCategory = 'ResLD' OR LoadCategory = 'ResHD'"
        execute_sql(sql, mu_path)
        sql = "UPDATE msm_Loadpoint SET loadflow = PerAreaLoad * ICIArea / 86400 WHERE LoadCategory = 'Commercial' OR LoadCategory = 'Industrial' OR LoadCategory = 'Institutional'"
        execute_sql(sql, mu_path)

        sql = "DROP TABLE New_Population"
        execute_sql(sql, mu_path)

    

In [5]:
#Generate BSF models
if generate_bsf:

    if not generate_future:
        mu_paths = []
        for f in os.listdir(output_folder):
            if f[-7:]==".sqlite" and not 'BSF' in f and not 'xADWF' in f:
                mu_paths.append(output_folder + '\\' + f)


    for mu_path_original in mu_paths:
                
        sqls = []

        pop_pos = mu_path_original.lower().find('pop')
        year = mu_path_original[pop_pos-4:pop_pos]

        for gwi in gwis:

            gwi_str = str(gwi).replace('.','p')

            mu_path = mu_path_original[:-7]  + "_BSF_" + gwi_str + "k.sqlite"
            os.remove(mu_path) if os.path.exists(mu_path) else None
            shutil.copyfile(mu_path_original, mu_path)
            
            print('Generating ' + mu_path)

            generate_script(script_path,mu_path)
            generate_mupp(mu_path_original,mu_path)

            sql = "SELECT muid, area, nettypeno FROM msm_Catchment WHERE nettypeno <> 2"
            catchments = sql_to_df(sql,mu_path)

            sqls = []
            for index, row in catchments.iterrows():
                MUID = str(row[0])
                Area = row[1]
                NetTypeNo = str(row[2])

                flow = Area * gwi

                sql = "UPDATE msm_LoadPoint SET MUID = '" + MUID + "_BSF', loadflow = " + str(flow / 86400 / 10000) + ", Description = 'BSF', LoadCategoryNo = 100, LoadCategory = 'BSF', LoadSubCategory = 'BSF_' & LoadLocation "
                sql += "WHERE MUID = '" + MUID + "_Load_8'"

                sqls.append(sql)

            execute_sql(sqls, mu_path)

            muid_new = model_area + "_BSF_" + gwi_str + "k_" + year + "pop_"
            
            sql =  "DELETE FROM msm_Project WHERE enable_catchment = 1"
            execute_sql(sql, mu_path)

            sql = "SELECT muid FROM msm_Project"
            sims = sql_to_df(sql,mu_path)
            
            for index, row in sims.iterrows():
                muid = row[0]
                if index == 0:
                    sql = "UPDATE msm_Project SET MUID = '" + muid_new + "', ScenarioName = 'Base', Description = 'BSF' WHERE MUID = '" + muid + "'"
                    execute_sql(sql, mu_path)
                    sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"          
                    execute_sql(sql, mu_path)
                else:
                    sql =  "DELETE FROM msm_Project WHERE simulationid = '" + muid + "'"
                    execute_sql(sql, mu_path)


In [9]:
if generate_xadwf:
    deficit_list = []
    diurnals = []

    if not generate_future:
        mu_paths = []
        for f in os.listdir(output_folder):
            if f[-7:]==".sqlite" and not 'BSF' in f and not 'xADWF' in f:
                mu_paths.append(output_folder + '\\' + f)

    for i, mu_path_original in enumerate(mu_paths):
                
        sqls = []

        pop_pos = mu_path_original.lower().find('pop')
        year = mu_path_original[pop_pos-4:pop_pos]

        sql = "SELECT Sum(loadflow) AS WaterLoad FROM msm_loadpoint WHERE loadcategory = 'Baseflow'"
        gwi_global = sql_to_df(sql,mu_path_original).iloc[0,0]
#         gwi_global = list(sql_to_df(sql,mu_path_original).WaterLoad)[0]
        

        sql = "SELECT ms_DPProfileD.ScheduleID AS Day_Type, ms_DPPatternD.Sqn AS [Hour], Sum(msm_Loadpoint.loadflow*ms_DPPatternD.DPValue) + " + str(gwi_global) + " AS Discharge "
        sql += "FROM ((msm_Loadpoint INNER JOIN msm_BBoundary ON msm_Loadpoint.LoadCategoryNo = msm_BBoundary.LoadCategoryNo) INNER JOIN ms_DPProfileD ON msm_BBoundary.DPProfileID = ms_DPProfileD.ProfileID) INNER JOIN ms_DPPatternD ON ms_DPProfileD.PatternID = ms_DPPatternD.PatternID "
        sql += "WHERE msm_Loadpoint.Active = 1 AND ms_DPProfileD.Active = 1 AND ms_DPPatternD.Active = 1 AND msm_BBoundary.Active = 1 "
        sql += "GROUP BY ms_DPProfileD.ScheduleID, ms_DPPatternD.Time "
        sql += "HAVING (LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekday' Or LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekend') AND ms_DPPatternD.Sqn <> 0 "
        sql += "ORDER BY scheduleid, time"
                        
        diurnal_wws = sql_to_df(sql,mu_path_original)        
        diurnal_wws.Hour = diurnal_wws.index
        diurnal_wws.loc[diurnal_wws.index > 23, 'HOUR'] = diurnal_wws.index[diurnal_wws.index > 23] - 24

        for times_adwf in times_adwf_list:
            
            mu_path = mu_path_original[:-7]  + "_" + str(times_adwf).replace('.','p') + "xADWF.sqlite"
            os.remove(mu_path) if os.path.exists(mu_path) else None
            shutil.copyfile(mu_path_original, mu_path)
            
            print('Generating ' + mu_path)

            generate_script(script_path,mu_path)
            generate_mupp(mu_path_original,mu_path)
            
            sqls = []

            sql = "SELECT muid FROM msm_Catchment"
            muids = list(sql_to_df(sql,mu_path).muid)
            for muid in muids:

                sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE CatchmentID = '" + muid + "' AND msm_Loadpoint.Active = 1"
                adwf = sql_to_df(sql,mu_path).iloc[0,0]

                sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE CatchmentID = '" + muid + "' AND LoadCategory = 'Baseflow' AND msm_Loadpoint.Active = 1"
                gwi = sql_to_df(sql,mu_path).iloc[0,0]

                sql = "SELECT Max(SumOfDPValue) "
                sql += "FROM (SELECT "
                sql += "ms_DPProfileD.ProfileID, Sum(msm_Loadpoint.loadflow*ms_DPPatternD.DPValue) AS SumOfDPValue "
                sql += "FROM ((msm_Loadpoint INNER JOIN msm_BBoundary ON msm_Loadpoint.LoadCategoryNo = msm_BBoundary.LoadCategoryNo) INNER JOIN ms_DPProfileD ON msm_BBoundary.DPProfileID = ms_DPProfileD.ProfileID) INNER JOIN ms_DPPatternD ON ms_DPProfileD.PatternID = ms_DPPatternD.PatternID "
                sql += "WHERE msm_Loadpoint.Active = 1 And msm_BBoundary.Active = 1 And ms_DPProfileD.Active = 1 And ms_DPPatternD.Active = 1 "
                sql += "GROUP BY ms_DPProfileD.ProfileID,ms_DPPatternD.MUID, msm_Loadpoint.CatchmentID, ms_DPProfileD.ScheduleID "
                sql += "HAVING msm_Loadpoint.CatchmentID = '" + muid + "' AND (LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekday' Or LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekend'))"
                
                pww = sql_to_df(sql,mu_path).iloc[0,0]
                pdwf = gwi + pww
                deficit = times_adwf * adwf - pdwf

                deficit_list.append([os.path.basename(mu_path),muid,deficit])

                sql = "UPDATE msm_Loadpoint SET muid = '" + muid + "_" + str(times_adwf).replace('.','p') + "xADWF', "
                sql += "loadflow = " + str(deficit) + ", Description = '" + str(times_adwf).replace('.','p') + "xADWF', "
                sql += "LoadCategoryNo = 1, LoadCategory = 'X-ADWF', LoadSubCategory = 'X-ADWF_' & LoadLocation "
                sql += "WHERE muid = '" + muid + "_Load_8' and msm_Loadpoint.Active = 1"
                sqls.append(sql)

            execute_sql(sqls, mu_path)
            
            muid_new = model_area + "_" + str(times_adwf).replace('.','p') + "ADWF_" + year + "pop_"
            sql =  "DELETE FROM msm_Project WHERE enable_catchment = 1"
            execute_sql(sql, mu_path)
            sql = "SELECT muid FROM msm_Project"
            sims = sql_to_df(sql,mu_path)
            for index, row in sims.iterrows():
                muid = row[0]
                if index == 0:
                    sql = "UPDATE msm_Project SET MUID = '" + muid_new + "', ScenarioName = 'Base', Description = 'BSF' WHERE MUID = '" + muid + "'"
                    execute_sql(sql, mu_path)
                    sql = "UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'"          
                    execute_sql(sql, mu_path)
                else:
                    sql =  "DELETE FROM msm_Project WHERE simulationid = '" + muid + "'"
                    execute_sql(sql, mu_path)

            sql = "SELECT Sum(loadflow) FROM msm_Loadpoint WHERE LoadCategory = 'X-ADWF' AND Active = 1"
            deficit = sql_to_df(sql,mu_path).iloc[0,0]
            
            diurnal_wws['Model'] = os.path.basename(mu_path)
            diurnal_wws['Deficit'] = deficit
            diurnal_wws = diurnal_wws[['Model','Day_Type','Hour','Discharge','Deficit']]
            
            if i == 0:
                diurnals_df = diurnal_wws.copy()
            else:
                diurnals_df = pd.concat([diurnals_df,diurnal_wws])           

    deficit_df = pd.DataFrame(deficit_list,columns=['Model','Catchment','Deficit'])
    deficit_df.to_csv(output_folder + '\\Deficits.csv',index=False)

    diurnals_df = pd.DataFrame(diurnals,columns=['Model','Day_Type','Hour','Discharge','Deficit'])
    diurnals_df.to_csv(output_folder + '\\X-ADWF_Diurnals.csv',index=False)
    
print('Done')

Generating J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\System_Assessment\Model_Generation\FSA_2030pop_V104_3xADWF.sqlite
Done


In [ ]:
if generate_sealed_vfd:
    
    if not generate_future:
        mu_paths = []
        for f in os.listdir(output_folder):
            if f[-7:]==".sqlite" and not 'VFD' in f:
                mu_paths.append(output_folder + '\\' + f)
 
    if seal_all and vfd_all:
        suffix = "S_V_"
        file_suffix = "_Sealed_VFD"
    elif seal_all and not vfd_all:
        suffix = "S_"
        file_suffix = "_Sealed"
    elif not seal_all and vfd_all:        
        suffix = "V_"
        file_suffix = "_VFD" 
    else:
        message = "Error! generate_sealed_vfd set to True but both vfd_all and seal_all set to False"
        raise ValueError(message)

    for mu_path_original in mu_paths:
                
        sqls = []

        mu_path = mu_path_original[:-7] + file_suffix + ".sqlite"
        os.remove(mu_path) if os.path.exists(mu_path) else None
        shutil.copyfile(mu_path_original, mu_path)
        
        generate_script(script_path,mu_path)
        generate_mupp(mu_path_original,mu_path)
        
        print('Generating ' + mu_path)

        if vfd_all == True:

            pumps_for_VFD = []
            pumps_turn_off = []

            sqls.append("INSERT INTO ms_Tab (muid,altid,active,description,typeno) SELECT 'Generic_Pump_Min',0,1,'Use for PS passing all inflow',2")
            sqls.append("INSERT INTO ms_Tab (muid,altid,active,description,typeno) SELECT 'Generic_Pump_Max',0,1,'Use for PS passing all inflow',2")

            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Max-1',0,1,'Generic_Pump_Max',1, 0, 50")
            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Max-2',0,1,'Generic_Pump_Max',2, 100, 50")
            
            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Min-1',0,1,'Generic_Pump_Min',1, 0, 0")
            sqls.append("INSERT INTO ms_TabD (muid,altid,active,tabid,sqn,value1,value2) SELECT 'Generic_Pump_Min-2',0,1,'Generic_Pump_Min',2, 100, 0")
            
            sql = "SELECT MUID FROM msm_Project WHERE enable_hd = 1"
            muids = list(sql_to_df(sql,mu_path).muid)
            for muid in muids:
                if len(muid) <= 40 - len(suffix):
                    muid_new = muid + suffix
                    sqls.append("UPDATE msm_Project SET muid = '" + muid_new + "' WHERE muid = '" + muid + "'")        
                    sqls.append("UPDATE msm_ProjectOutput SET simulationid = '" + muid_new + "' WHERE simulationid = '" + muid + "'")            
                                
            sql = "SELECT assetname, muid, startlevel from msm_Pump ORDER BY assetname, startlevel"
            pumps = sql_to_df(sql,mu_path)

            previous_asset = 'xxxx'
            for index, row in pumps.iterrows():
                asset = str(row[0])
                if not asset in excluded_asset_names and asset != 'None':
                    muid = str(row[1])
                    if previous_asset != asset:
                        pumps_for_VFD.append(muid)
                    else:
                        pumps_turn_off.append (muid)
                    previous_asset = asset
                else:
                    print ('Skipped ' + str(row[1]))

            for pump_for_VFD in pumps_for_VFD:
                sqls.append("UPDATE msm_Pump SET qmaxsetid = 'Generic_Pump_Max',  qminsetid = 'Generic_Pump_Min', speedno = 2, captypeno = 2, controltypeno = 1, startlevel = stoplevel + 0.1,  wetwellsetpoint = stoplevel + 0.1  WHERE muid = '" + pump_for_VFD + "'")
                sqls.append("UPDATE msm_RTC SET applyno = 0 WHERE pumpid = '" + pump_for_VFD + "'")

            for pump_turn_off in pumps_turn_off:
                sqls.append("UPDATE msm_Pump SET startlevel = startlevel + 100, stoplevel = stoplevel + 100,  controltypeno = 1 WHERE MUID = '" + pump_turn_off + "'")
                sqls.append("UPDATE msm_RTC SET applyno = 0 WHERE pumpid = '" + pump_turn_off + "'")

        execute_sql(sqls, mu_path)
